In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
from dpgo_bag_eval import *
from simulate_utils import *
import numpy as np
import os

def generate_4dof_noise(pg, pos_noise=1.0, rot_noise=0.2):
    agent_offsets = np.random.normal(0, pos_noise*3, (len(pg.agents), 4))
    agent_offsets[:,3] = np.random.normal(0, rot_noise*2, len(pg.agents))
    agent_offsets[0,:] = 0
    for _id in pg.keyframes:
        if _id != 0:
            kf = pg.keyframes[_id]
            agent_offset = agent_offsets[kf.agent_id, :]
            kf.pos = kf.pos + np.random.normal(0, pos_noise, 3) + agent_offset[0:3]
            yaw_noise = np.random.normal(0, rot_noise)
            quat_noise = quaternion_from_euler(0, 0, yaw_noise)
            kf.quat = quaternion_multiply(kf.quat, quat_noise)
        

def generate_data(data_file, data_folder, agent_num, partition="id", show=False, is_4dof=True, pos_noise=1.0, rot_noise=0.2, gen_from_gt=False):
    pg:PoseGraph = PoseGraph()
    isFile = os.path.isfile(data_file)
    if isFile:
        pg.read_g2o_single(data_file)
        pg.write_to_g2o(f"{data_folder}/raw.g2o")
    else:
        pg.read_g2o_folder(data_file)
    try:
        pg_gt:PoseGraph = generate_groundtruth_sesync(pg, "/tmp/sesync_gt")
    except Exception as e:
        print("[Warn] Cannot generate groundtruth:", e)
        raise e
        pg_gt = None
    if pg_gt:
        for kf_id in pg_gt.keyframes:
            kf = pg_gt.keyframes[kf_id]
    if gen_from_gt and pg_gt:
            pg_noise = copy.deepcopy(pg_gt)
    else:
        pg_noise = copy.deepcopy(pg)
    if is_4dof:
        pg_noise.convert_to_4dof()
        if pg_gt:
            pg_gt.convert_to_4dof()
    if isFile:
        partitioning(pg_noise, "vol", agent_num=agent_num, show=False, method=partition)
        generate_4dof_noise(pg_noise, pos_noise=pos_noise, rot_noise=rot_noise)
        pg_noise.update_edges()
    pg_noise.write_to_g2o_folder(f"{data_folder}/input/", cvt_id=True)
    if show:
        pg.show("raw", show_edges=False)
        if pg_gt:
            pg_gt.show("GT", clear=False, show_edges=False)
        pg_noise.show("noise", show_edges=False, clear=False)
    partitioning(pg_noise, "vol", agent_num=agent_num, show=False, method="union")
    pg_noise.update_edges()
    pg_noise.write_to_g2o(f"{data_folder}/input.g2o")
    if pg_gt:
        pg_gt.write_to_g2o(f"{data_folder}/groundtruth.g2o")



In [ ]:
input_data = "/home/xuhao/data/ral2021/raw/parking-garage.g2o"
output_folder = "/home/xuhao/data/d2slam/pgo/parking-garage"
generate_data(input_data, output_folder, 5, partition="id", show=True, is_4dof=False)

In [ ]:
input_data = "/home/xuhao/data/ral2021/raw/input_INTEL_g2o.g2o"
output_folder = "/home/xuhao/data/d2slam/pgo/input_INTEL_g2o"
generate_data(input_data, output_folder, 5, partition="id", show=True, is_4dof=True)

In [ ]:
input_data = "/home/xuhao/data/ral2021/raw/cubicle.g2o"
output_folder = "/home/xuhao/data/d2slam/pgo/cubicle"
generate_data(input_data, output_folder, 5, partition="id", show=True, is_4dof=True)

In [ ]:
input_data = "/home/xuhao/data/ral2021/raw/torus3D.g2o"
output_folder = "/home/xuhao/data/d2slam/pgo/torus3D"
generate_data(input_data, output_folder, 5, partition="id", show=True, is_4dof=False, pos_noise=0.5, rot_noise=0.0)

In [ ]:
input_data = "/home/xuhao/data/d2slam/pgo/tum_corr/raw"
output_folder = "/home/xuhao/data/d2slam/pgo/tum_corr/"
generate_data(input_data, output_folder, 2, show=True, is_4dof=True, pos_noise=0.0, rot_noise=0.0)